# 📘 Dependency Inversion Principle (DIP)

## Definition

**Don't hardcode dependencies inside your classes. Instead, let other classes pass in what they need.**

Think of it like this: instead of a class creating its own tools, it should receive the tools it needs from the outside. This makes your code more flexible and easier to test.

## 🎯 What This Means in Simple Terms

### **The Problem:**
When a class creates its own dependencies, it becomes "stuck" to those specific implementations. It's like a chef who only knows how to cook with one specific brand of knife - if that knife breaks, the chef can't cook anymore.

### **The Solution:**
Instead of creating dependencies inside the class, pass them in from the outside. This is like giving the chef any knife that can cut - they can work with different brands, different types, even different tools as long as they can cut.

### **Why This Matters:**
- **Testing**: You can easily test your code by passing in "fake" versions of dependencies
- **Flexibility**: You can swap out implementations without changing your main code
- **Maintainability**: Changes to one part don't break other parts



## ❌ Bad Example (Violates DIP)

Let's look at a class that violates the Dependency Inversion Principle:


In [1]:
# ❌ BAD EXAMPLE: High-level module depends on low-level modules
# =============================================================================
# This example shows what NOT to do - it violates the Dependency Inversion Principle
# =============================================================================

# These are "low-level" classes - they handle specific technical details
class FileLogger:
    """Low-level module: handles file logging."""
    def log(self, message):
        with open("app.log", "a") as f:
            f.write(f"{message}\n")
        print(f"Logged to file: {message}")

class EmailService:
    """Low-level module: handles email sending."""
    def send_email(self, to, subject, body):
        print(f"Email sent to {to}: {subject} - {body}")

class DatabaseService:
    """Low-level module: handles database operations."""
    def save_user(self, user_data):
        print(f"User saved to database: {user_data}")

# This is the "high-level" class - it should coordinate business logic
class UserService:
    """High-level module: depends directly on low-level modules."""
    def __init__(self):
        # ❌ PROBLEM 1: HARDCODED DEPENDENCIES
        # The UserService creates its own dependencies instead of receiving them
        # This makes it "stuck" to these specific implementations
        
        # ❌ PROBLEM 2: TIGHT COUPLING
        # UserService knows about FileLogger, EmailService, DatabaseService
        # If we want to change any of these, we must modify UserService
        
        # ❌ PROBLEM 3: HARD TO TEST
        # We can't easily test UserService with "fake" versions of these classes
        # Every test will actually write to files, send emails, and save to database
        
        # ❌ PROBLEM 4: VIOLATES DIP
        # High-level module (UserService) depends on low-level modules (FileLogger, etc.)
        # It should depend on abstractions instead!
        
        self.logger = FileLogger()           # Hardcoded to file logging
        self.email_service = EmailService()  # Hardcoded to this email service
        self.database = DatabaseService()    # Hardcoded to this database service
    
    def register_user(self, user_data):
        # High-level business logic - this is what the class should focus on
        print("Registering user...")
        
        # Direct calls to low-level modules
        # ❌ PROBLEM: UserService knows HOW logging, email, and database work
        # It should only know WHAT it needs to do, not HOW to do it
        self.logger.log(f"User registration started: {user_data['name']}")
        self.database.save_user(user_data)
        self.email_service.send_email(
            user_data['email'], 
            "Welcome!", 
            f"Welcome {user_data['name']}!"
        )
        self.logger.log(f"User registration completed: {user_data['name']}")

# Example usage
user_data = {
    'name': 'John Doe',
    'email': 'john@example.com',
    'age': 30
}

user_service = UserService()
user_service.register_user(user_data)

# =============================================================================
# WHAT'S WRONG WITH THIS APPROACH?
# =============================================================================
# 1. HARDCODED DEPENDENCIES: UserService creates its own tools
# 2. TIGHT COUPLING: Can't change implementations without modifying UserService
# 3. HARD TO TEST: Can't use fake versions for testing
# 4. VIOLATES DIP: High-level depends on low-level (should be the opposite)
# 5. INFLEXIBLE: What if we want to use a different logger or email service?
# =============================================================================


Registering user...
Logged to file: User registration started: John Doe
User saved to database: {'name': 'John Doe', 'email': 'john@example.com', 'age': 30}
Email sent to john@example.com: Welcome! - Welcome John Doe!
Logged to file: User registration completed: John Doe


### 🔴 Why is this bad?

The `UserService` class violates DIP because:

1. **Direct dependencies**: High-level `UserService` directly depends on low-level concrete classes
2. **Tight coupling**: Changes to low-level modules require changes to high-level modules
3. **Hard to test**: Cannot easily mock dependencies for unit testing
4. **Inflexible**: Cannot swap implementations without modifying `UserService`
5. **Violates abstraction**: High-level module knows about implementation details

**Problems:**
- If we want to use a different logger (e.g., database logger), we must modify `UserService`
- If we want to use a different email service, we must modify `UserService`
- Testing is difficult because we can't easily mock the dependencies
- The high-level business logic is tightly coupled to low-level implementation details


## ✅ Good Example (Follows DIP)

Let's fix this by introducing abstractions and dependency injection:

## 🔧 What Are Abstractions? (ABC and abstractmethod)

Before we see the solution, let's understand the tools we'll use:

### **ABC (Abstract Base Class):**
- `ABC` stands for "Abstract Base Class"
- It's a special type of class that cannot be instantiated directly
- Think of it as a "template" or "contract" that other classes must follow
- It's like a blueprint for a house - you can't live in the blueprint, but you can build houses from it

### **abstractmethod:**
- `@abstractmethod` is a decorator that marks a method as "must be implemented"
- Any class that inherits from an ABC with abstract methods MUST implement those methods
- It's like saying "any class that wants to be a Logger MUST have a log() method"

### **Why Use These?**
- **Contracts**: They define what methods a class must have
- **Flexibility**: You can swap different implementations as long as they follow the contract
- **Safety**: Python will give you an error if you forget to implement required methods

### **Simple Example:**
```python
from abc import ABC, abstractmethod

class Animal(ABC):  # This is an abstract base class
    @abstractmethod
    def make_sound(self):  # Any Animal MUST have this method
        pass

class Dog(Animal):
    def make_sound(self):  # Dog implements the required method
        return "Woof!"

class Cat(Animal):
    def make_sound(self):  # Cat implements the required method
        return "Meow!"

# This works:
dog = Dog()
print(dog.make_sound())  # "Woof!"

# This would cause an error:
# animal = Animal()  # Error! Can't create abstract class directly
```

### **Complete Example with High-Level Class:**
Now let's see how a high-level class can use these low-level classes:

```python
class AnimalTrainer:
    """High-level class that works with any Animal"""
    
    def __init__(self, animal: Animal):  # Accepts any Animal (Dog, Cat, etc.)
        self.animal = animal
    
    def train_animal(self):
        """Train the animal to make sounds"""
        print(f"Training animal...")
        sound = self.animal.make_sound()  # Works with any Animal!
        print(f"Animal says: {sound}")
        print("Training complete!")

# Usage examples:
dog_trainer = AnimalTrainer(Dog())  # Pass in a Dog
dog_trainer.train_animal()
# Output: Training animal...
#         Animal says: Woof!
#         Training complete!

cat_trainer = AnimalTrainer(Cat())  # Pass in a Cat
cat_trainer.train_animal()
# Output: Training animal...
#         Animal says: Meow!
#         Training complete!
```

### **Why This is Powerful:**
- **Flexibility**: `AnimalTrainer` can work with ANY animal that follows the `Animal` contract
- **Easy Testing**: You can create a "fake" animal for testing
- **Extensibility**: Add new animals (Bird, Fish) without changing `AnimalTrainer`
- **No Hardcoding**: `AnimalTrainer` doesn't know about specific animal types

Now let's apply this same pattern to our DIP solution:


In [2]:
from abc import ABC, abstractmethod

# =============================================================================
# STEP 1: CREATE ABSTRACTS (CONTRACTS)
# =============================================================================
# These are the "contracts" that define what methods classes must have.
# Think of them as blueprints that other classes must follow.

class Logger(ABC):
    """Abstraction for logging functionality."""
    @abstractmethod
    def log(self, message):
        pass

class EmailService(ABC):
    """Abstraction for email functionality."""
    @abstractmethod
    def send_email(self, to, subject, body):
        pass

class UserRepository(ABC):
    """Abstraction for user data persistence."""
    @abstractmethod
    def save_user(self, user_data):
        pass

# =============================================================================
# STEP 2: CREATE CONCRETE IMPLEMENTATIONS (LOW-LEVEL CLASSES)
# =============================================================================
# These classes implement the contracts. They are the "workers" that do the actual work.
# Notice how each class implements the required methods from the contracts.

# Logger implementations
class FileLogger(Logger):
    """Concrete implementation: file logging."""
    def log(self, message):
        with open("app.log", "a") as f:
            f.write(f"{message}\n")
        print(f"Logged to file: {message}")

class DatabaseLogger(Logger):
    """Concrete implementation: database logging."""
    def log(self, message):
        print(f"Logged to database: {message}")

class ConsoleLogger(Logger):
    """Concrete implementation: console logging."""
    def log(self, message):
        print(f"Console log: {message}")

# Email service implementations
class SMTPEmailService(EmailService):
    """Concrete implementation: SMTP email service."""
    def send_email(self, to, subject, body):
        print(f"SMTP email sent to {to}: {subject} - {body}")

class SendGridEmailService(EmailService):
    """Concrete implementation: SendGrid email service."""
    def send_email(self, to, subject, body):
        print(f"SendGrid email sent to {to}: {subject} - {body}")

# User repository implementations
class MySQLUserRepository(UserRepository):
    """Concrete implementation: MySQL database."""
    def save_user(self, user_data):
        print(f"User saved to MySQL: {user_data}")

class PostgreSQLUserRepository(UserRepository):
    """Concrete implementation: PostgreSQL database."""
    def save_user(self, user_data):
        print(f"User saved to PostgreSQL: {user_data}")

# =============================================================================
# STEP 3: CREATE HIGH-LEVEL CLASS (DEPENDS ON ABSTRACTS)
# =============================================================================
# This is the "boss" class that coordinates everything.
# It depends on the contracts (abstractions), not the concrete implementations.
# This is the key to DIP!

class UserService:
    """High-level module: depends only on abstractions."""
    def __init__(self, logger: Logger, email_service: EmailService, user_repository: UserRepository):
        # ✅ DEPENDENCY INJECTION: We receive the dependencies instead of creating them
        # ✅ DEPENDS ON ABSTRACTS: We depend on Logger, EmailService, UserRepository (contracts)
        # ✅ NOT CONCRETE: We don't know about FileLogger, SMTPEmailService, etc.
        self.logger = logger
        self.email_service = email_service
        self.user_repository = user_repository
    
    def register_user(self, user_data):
        # High-level business logic - this is what the class is really about
        print("Registering user...")
        
        # Uses abstractions - doesn't know about concrete implementations
        # The class just calls the methods, it doesn't care HOW they work
        self.logger.log(f"User registration started: {user_data['name']}")
        self.user_repository.save_user(user_data)
        self.email_service.send_email(
            user_data['email'], 
            "Welcome!", 
            f"Welcome {user_data['name']}!"
        )
        self.logger.log(f"User registration completed: {user_data['name']}")

# =============================================================================
# STEP 4: DEMONSTRATE THE FLEXIBILITY
# =============================================================================
# Now we can easily swap different implementations without changing UserService!
# This is the power of DIP - the high-level class is flexible and testable.

user_data = {
    'name': 'John Doe',
    'email': 'john@example.com',
    'age': 30
}

# Configuration 1: File logging + SMTP email + MySQL database
print("Configuration 1:")
user_service_1 = UserService(
    logger=FileLogger(),           # Pass in a FileLogger
    email_service=SMTPEmailService(),  # Pass in SMTP email service
    user_repository=MySQLUserRepository()  # Pass in MySQL repository
)
user_service_1.register_user(user_data)

print("\nConfiguration 2:")
# Configuration 2: Console logging + SendGrid email + PostgreSQL database
user_service_2 = UserService(
    logger=ConsoleLogger(),        # Pass in a ConsoleLogger
    email_service=SendGridEmailService(),  # Pass in SendGrid email service
    user_repository=PostgreSQLUserRepository()  # Pass in PostgreSQL repository
)
user_service_2.register_user(user_data)


Configuration 1:
Registering user...
Logged to file: User registration started: John Doe
User saved to MySQL: {'name': 'John Doe', 'email': 'john@example.com', 'age': 30}
SMTP email sent to john@example.com: Welcome! - Welcome John Doe!
Logged to file: User registration completed: John Doe

Configuration 2:
Registering user...
Console log: User registration started: John Doe
User saved to PostgreSQL: {'name': 'John Doe', 'email': 'john@example.com', 'age': 30}
SendGrid email sent to john@example.com: Welcome! - Welcome John Doe!
Console log: User registration completed: John Doe


### 🟢 Why is this good?

The refactored design follows DIP because:

1. **Abstractions**: High-level and low-level modules both depend on abstractions
2. **Dependency injection**: Dependencies are injected rather than created internally
3. **Flexibility**: Easy to swap implementations without changing high-level code
4. **Testability**: Can easily mock dependencies for unit testing
5. **Loose coupling**: High-level modules don't know about concrete implementations

**Benefits:**
- ✅ **Easy to test**: Can inject mock objects for testing
- ✅ **Flexible configuration**: Can use different implementations in different environments
- ✅ **Maintainable**: Changes to low-level modules don't affect high-level modules
- ✅ **Extensible**: Easy to add new implementations without changing existing code


## 🧪 Testing with Dependency Injection

Let's see how DIP makes testing much easier:


In [3]:
# Mock implementations for testing
class MockLogger(Logger):
    """Mock logger for testing - captures log messages."""
    def __init__(self):
        self.log_messages = []
    
    def log(self, message):
        self.log_messages.append(message)
        print(f"Mock log: {message}")

class MockEmailService(EmailService):
    """Mock email service for testing - captures sent emails."""
    def __init__(self):
        self.sent_emails = []
    
    def send_email(self, to, subject, body):
        email = {'to': to, 'subject': subject, 'body': body}
        self.sent_emails.append(email)
        print(f"Mock email sent: {email}")

class MockUserRepository(UserRepository):
    """Mock user repository for testing - captures saved users."""
    def __init__(self):
        self.saved_users = []
    
    def save_user(self, user_data):
        self.saved_users.append(user_data)
        print(f"Mock user saved: {user_data}")

# Test function using dependency injection
def test_user_registration():
    """Test user registration with mock dependencies."""
    print("Testing user registration with mocks:\n")
    
    # Create mock dependencies
    mock_logger = MockLogger()
    mock_email_service = MockEmailService()
    mock_user_repository = MockUserRepository()
    
    # Inject mocks into UserService
    user_service = UserService(
        logger=mock_logger,
        email_service=mock_email_service,
        user_repository=mock_user_repository
    )
    
    # Test data
    test_user = {
        'name': 'Test User',
        'email': 'test@example.com',
        'age': 25
    }
    
    # Execute the method
    user_service.register_user(test_user)
    
    # Verify the results
    print(f"\nVerification:")
    print(f"Log messages captured: {len(mock_logger.log_messages)}")
    for msg in mock_logger.log_messages:
        print(f"  - {msg}")
    
    print(f"Emails sent: {len(mock_email_service.sent_emails)}")
    for email in mock_email_service.sent_emails:
        print(f"  - To: {email['to']}, Subject: {email['subject']}")
    
    print(f"Users saved: {len(mock_user_repository.saved_users)}")
    for user in mock_user_repository.saved_users:
        print(f"  - {user}")
    
    # Assertions
    assert len(mock_logger.log_messages) == 2, "Should log start and completion"
    assert len(mock_email_service.sent_emails) == 1, "Should send one welcome email"
    assert len(mock_user_repository.saved_users) == 1, "Should save one user"
    assert mock_user_repository.saved_users[0] == test_user, "Saved user should match input"
    
    print("\n✅ All tests passed!")

# Run the test
test_user_registration()


Testing user registration with mocks:

Registering user...
Mock log: User registration started: Test User
Mock user saved: {'name': 'Test User', 'email': 'test@example.com', 'age': 25}
Mock email sent: {'to': 'test@example.com', 'subject': 'Welcome!', 'body': 'Welcome Test User!'}
Mock log: User registration completed: Test User

Verification:
Log messages captured: 2
  - User registration started: Test User
  - User registration completed: Test User
Emails sent: 1
  - To: test@example.com, Subject: Welcome!
Users saved: 1
  - {'name': 'Test User', 'email': 'test@example.com', 'age': 25}

✅ All tests passed!


## 🔄 Another DIP Example: Shape Area Calculator

Let's look at another example with shape calculations:


In [4]:
# ❌ Bad Example: Violates DIP
class ConsoleOutput:
    """Low-level module: console output."""
    def display(self, message):
        print(f"Console: {message}")

class FileOutput:
    """Low-level module: file output."""
    def write(self, message):
        with open("output.txt", "a") as f:
            f.write(f"{message}\n")

class AreaCalculator:
    """High-level module: depends directly on low-level modules."""
    def __init__(self):
        # ❌ Problem: Direct dependency on concrete implementations
        self.console = ConsoleOutput()
        self.file = FileOutput()
    
    def calculate_and_display_area(self, shapes):
        total_area = sum([shape.area() for shape in shapes])
        
        # Direct calls to low-level modules
        self.console.display(f"Total area: {total_area}")
        self.file.write(f"Total area: {total_area}")
        
        return total_area

# ✅ Good Example: Follows DIP
class OutputService(ABC):
    """Abstraction for output functionality."""
    @abstractmethod
    def output(self, message):
        pass

class ConsoleOutputService(OutputService):
    """Concrete implementation: console output."""
    def output(self, message):
        print(f"Console: {message}")

class FileOutputService(OutputService):
    """Concrete implementation: file output."""
    def output(self, message):
        with open("output.txt", "a") as f:
            f.write(f"{message}\n")

class DatabaseOutputService(OutputService):
    """Concrete implementation: database output."""
    def output(self, message):
        print(f"Database: {message}")

class AreaCalculator:
    """High-level module: depends only on abstraction."""
    def __init__(self, output_service: OutputService):
        # ✅ Dependency injection: depends on abstraction
        self.output_service = output_service
    
    def calculate_and_display_area(self, shapes):
        total_area = sum([shape.area() for shape in shapes])
        
        # Uses abstraction - doesn't know about concrete implementation
        self.output_service.output(f"Total area: {total_area}")
        
        return total_area

# Example usage
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def area(self):
        return self.width * self.height

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    
    def area(self):
        return 3.14 * self.radius * self.radius

# Test with different output services
shapes = [Rectangle(5, 3), Circle(4)]

print("Using console output:")
calculator_console = AreaCalculator(ConsoleOutputService())
calculator_console.calculate_and_display_area(shapes)

print("\nUsing file output:")
calculator_file = AreaCalculator(FileOutputService())
calculator_file.calculate_and_display_area(shapes)

print("\nUsing database output:")
calculator_db = AreaCalculator(DatabaseOutputService())
calculator_db.calculate_and_display_area(shapes)


Using console output:
Console: Total area: 65.24000000000001

Using file output:

Using database output:
Database: Total area: 65.24000000000001


65.24000000000001

## 🎯 Key Takeaways

### Dependency Inversion Principle Summary:

1. **Depend on abstractions**: High-level modules should depend on interfaces, not concrete classes
2. **Dependency injection**: Inject dependencies rather than creating them internally
3. **Inversion of control**: Let external code control the dependencies
4. **Loose coupling**: High-level and low-level modules are decoupled through abstractions
5. **Flexibility**: Easy to swap implementations without changing high-level code

### When DIP is violated:
- High-level modules directly instantiate low-level modules
- Classes create their own dependencies internally
- Hard to test because dependencies can't be mocked
- Difficult to change implementations without modifying high-level code

### How to apply DIP:
- **Create abstractions**: Define interfaces for dependencies
- **Use dependency injection**: Pass dependencies through constructor or methods
- **Avoid `new` in business logic**: Don't create objects inside business methods
- **Use dependency injection containers**: For complex dependency management

### Benefits of DIP:
- ✅ **Easy testing**: Can inject mock objects for unit testing
- ✅ **Flexible configuration**: Different implementations for different environments
- ✅ **Maintainable code**: Changes to low-level modules don't affect high-level modules
- ✅ **Extensible design**: Easy to add new implementations

### Design patterns that help with DIP:
- **Dependency Injection**: Inject dependencies rather than creating them
- **Factory Pattern**: Create objects without specifying exact classes
- **Service Locator**: Find and inject services
- **Inversion of Control (IoC) containers**: Manage dependencies automatically

### Remember:
> "Depend on abstractions, not concretions" - Robert C. Martin
